### Examine a SWMM report file

In [ ]:
from swmm_read import *
#fname = "wingohocking_PWD_2012-13.rpt"
fname = "SWMM_modified.rpt"
rptFileStr = read_rpt_file(fname)
#print rptFileStr

### Read and display the report

In [ ]:
reportDict = read_report(rptFileStr)
reportDict

### Test SWMM runs from Jupyter 

In [ ]:
from swmm_run import *
swmmInpFileNameStr = "wingohocking_PWD_2011-12.inp"  
swmmRptFileNameStr = "wingohocking_PWD_2011-12.rpt"
timeStatsDict = runswmm(swmmInpFileNameStr,swmmRptFileNameStr)

### Select a watershed, a weather year, and find the best treatment ratio

In [ ]:
from swmm_read import *
from swmm_run import *
swmmRptFileNameStr = "wingohocking_PWD_2012-13.rpt"
swmmRptStr = read_rpt_file(swmmRptFileNameStr)
swmmRptDict = read_report(swmmRptStr)
outfallFlowTimeSeries = swmmRptDict["outflow_series"]
wingohockingImperviousArea = 3122.0  # acres
#ratioList = [0.075,0.080,0.085,0.090,0.095,0.10]
ratioList = [0.080,0.081,0.082,0.083,0.084,0.085]
for treatmentRatio in ratioList:   # CFS / impervious acre
    thresholdCFS = wingohockingImperviousArea*treatmentRatio
    outfallDict = calculate_cso(outfallFlowTimeSeries,thresholdCFS)
    csoVolumeMgal = outfallDict['cso_volume']/1e6
    totVolumeMgal = outfallDict['tot_volume']/1e6
    print "Treatment Ratio = %0.3f,  CSO Volume = %0.0f MGAL,  Total Volume = %0.0f MGAL" % (treatmentRatio,csoVolumeMgal,totVolumeMgal)

### SINGLE RUN: Scatter LID around subcatchments, calculate CSO, display results

In [ ]:
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import *

squareFeetInOneAcre = 43560.0
swmmInpFileNameStr = "wingohocking_PWD_2012-13.inp"  # initial SWMM .inp file
treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
watershedImpervAreaAcre = total_impervious_area(swmmInpFileNameStr)
thresholdOutflowCFS = treatmentRatio*watershedImpervAreaAcre
lidDictFileNameStr = "LID_Dict.yaml"
lidShedsFileNameStr = "Wingohocking_LID_Sheds_List.yaml"
f = open(lidDictFileNameStr,'r')   # Read the no LID changable parameters from file
lidDict = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
f = open(lidShedsFileNameStr,'r')   # Read the no LID changable parameters from file
lidShedsList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()

swmmInpStr = read_inp_file(swmmInpFileNameStr)
(section_names,sections) = read_inp(swmmInpStr)
model = swmm_model("Model",section_names,sections)
swmmInputFileStr=model.output()

subcatAreaDict = {}
subcatImpPctDict = {}
subcatTreatableAreaDict = {}
for lidShed in lidShedsList:
    area = model.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = area
    pctImperv = model.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = area*pctImperv/100.0

lid = 'wakefield_BR_RG'

contribImpervAreaAcre = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
subcatMaxLID = {}
for lidShed in lidShedsList:
    maxLID = int(subcatTreatableAreaDict[lidShed]/contribImpervAreaAcre)
    subcatMaxLID[lidShed] = maxLID
    model.lidChangeNumber(lidShed,lid,maxLID)
    area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment (all subcats the same now)
    model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre)  # must call to properly set FromImp parameter !!
swmmInputFileStr=model.output()
swmmRptDict = swmm_run_return_results(swmmInputFileStr)
outfallFlowTimeSeries = swmmRptDict["outflow_seriesCFS"]
outfallDict = calculate_cso(outfallFlowTimeSeries,thresholdOutflowCFS)
csoVolumeMgal = outfallDict['cso_volume']/1e6
totVolumeMgal = outfallDict['tot_volume']/1e6
csoReductionMgal = 1552.0 - csoVolumeMgal
print "Treatment Ratio = %0.3f,  CSO Volume = %0.0f MGAL,  CSO Reduction = %0.0f MGAL" % (treatmentRatio,csoVolumeMgal,csoReductionMgal)

### SWMM input and report files associated with the single run are stored and can be displayed using "print"

In [ ]:
str = swmmRptDict['swmmInpFileStr']
#str = swmmRptDict['swmmRptFileStr']
print str

### MULTIPLE RUNS: Import Libraries, Specify SWMM INP file, Setup MongoDB Database to store results

In [ ]:
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import *
from pymongo import MongoClient
from random import randint

squareFeetInOneAcre = 43560.0
swmmInpFileNameStr = "wingohocking_PWD_2012-13.inp"  # initial SWMM .inp file
treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
lidDictFileNameStr = "LID_Dict.yaml"
lidShedsFileNameStr = "Wingohocking_LID_Sheds_List.yaml"

watershedImpervAreaAcre = total_impervious_area(swmmInpFileNameStr)
thresholdOutflowCFS = treatmentRatio*watershedImpervAreaAcre

f = open(lidDictFileNameStr,'r')   # Read the no LID changable parameters from file
lidDict = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
f = open(lidShedsFileNameStr,'r')   # Read the no LID changable parameters from file
lidShedsList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()

swmmInpStr = read_inp_file(swmmInpFileNameStr)
(section_names,sections) = read_inp(swmmInpStr)
model = swmm_model("Model",section_names,sections)
subcatAreaDict = {}
subcatImpPctDict = {}
subcatTreatableAreaDict = {}
for lidShed in lidShedsList:
    area = model.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = area
    pctImperv = model.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = area*pctImperv/100.0

mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"

client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionName = 'y17m04d22_45_Subcat_Testing_Only'
thisRunCollection = db[collectionName]    # use or create the collection

### Execute the Multirun Case after Zero Run Case is run BELOW

In [ ]:
lid = 'wakefield_BR_RG'

contribImpervAreaAcre = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
subcatMaxLID = {}
for lidShed in lidShedsList:
    maxLID = int(subcatTreatableAreaDict[lidShed]/contribImpervAreaAcre)
    subcatMaxLID[lidShed] = maxLID
    model.lidChangeNumber(lidShed,lid,maxLID)
    area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment (all subcats the same now)
    model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre)  # must call to properly set FromImp parameter !!
swmmInputFileStr=model.output()
#print swmmInputFileStr
swmmRptDict = swmm_run_return_results(swmmInputFileStr)
outfallFlowTimeSeries = swmmRptDict["outflow_seriesCFS"]
outfallDict = calculate_cso(outfallFlowTimeSeries,thresholdOutflowCFS)
csoVolumeMgal = outfallDict['cso_volume']/1e6
totVolumeMgal = outfallDict['tot_volume']/1e6
csoReductionMgal = 1552.0 - csoVolumeMgal
print "Treatment Ratio = %0.3f,  CSO Volume = %0.0f MGAL,  CSO Reduction = %0.0f MGAL" % (treatmentRatio,csoVolumeMgal,csoReductionMgal)

In [ ]:
# Run or continue a run:
%autoreload
import yaml
from swmm_run import *
from swmm_objects import *
from swmm_read import *
from copy import deepcopy

from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
swmmInpFile = "wingohocking_PWD_Wakefield.inp"  # initial SWMM .inp file
lidListFile = "PWD_wingo_Wakefield_LID_List.yaml"
################################################
squareFeetInOneAcre = 43560.0
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y17m01d10_AEM_45_Subcat_Wakefield_Rain_Garden_1'
thisRunCollection = db[collection]
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
(section_names,sections) = read_inp(swmmInpStr)
modelTemp = swmm_model("ModelTemp",section_names,sections)
f = open(lidListFile,'r')   # Read the LID parameters used to calculate [LID_USAGE]
lidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
#########################
numSubcatch = 45
numLID = 1
numberOfSimulations = 50
#########################
numVariables = numSubcatch*numLID
initialRemainingTreatableArea = {}   # Dict indexed by subcatchment name string 
for lidListEntry in range(numVariables):
    thisLidInfo = lidList[lidListEntry]
    subcatName = thisLidInfo['Subcat']
    areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres   
    impPctSubcat = modelTemp.getSubcatchmentPctImperv(subcatName)        
    initialRemainingTreatableArea[subcatName] = (impPctSubcat/100.0)*areaSubcat  # INITIAL values (Acre)
#print initialRemainingTreatableArea
permList = []
greenedAcreList = []

for i in range(0,numberOfSimulations):#number of simulations
    remainingTreatableArea = dict(initialRemainingTreatableArea)   # Dict indexed by subcatchment name string  
    lidListEntry = 0
    randomPerm = []
    greened = []
    lidListEntry = 0    
    for lid in range(numLID):
        for subcat in range(numSubcatch):
            thisLidInfo = lidList[lidListEntry]
            #lidName = thisLidInfo['LID']
            subcatName = thisLidInfo['Subcat']
            areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres          
            contribImpervArea = thisLidInfo['ContribImpervArea']/squareFeetInOneAcre  #ft2 to Acres        
            numLidMaximum = int(remainingTreatableArea[subcatName]/contribImpervArea)
            #randomIntegerUpper = randint(1,19)
            #upperFraction = float(randomIntegerUpper)/20.0

            #randomScaleFactor = randint(3,50)
            #random1 = randint(1,randomScaleFactor)
            #random2 = randint(1,randomScaleFactor)
            #if random1 > random2:
            #    randomUpper = random1
            #    randomLower = random2
            #else:
            #    randomUpper = random2
            #    randomLower = random1
            ############
            #lower = numLidMaximum/randomUpper
            #upper = numLidMaximum/randomLower 
            ############
            fixedLowerFraction = 0.7
            fixedUpperFraction = 0.98
            lower = int(fixedLowerFraction*numLidMaximum)
            upper = int(fixedUpperFraction*numLidMaximum)
            ############
            randomNumberLid = randint(lower,upper)
            randomPerm.append(randomNumberLid)
            greened.append(randomNumberLid*lidList[lidListEntry]['GreenedAcres'])
            remainingTreatableArea[subcatName] = remainingTreatableArea[subcatName] - \
                  randomNumberLid*contribImpervArea #Acres
            lidListEntry += 1
        #print remainingTreatableArea
    permList.append(randomPerm)
    greenedAcreList.append(greened)
#print permList 
#print greenedAcreList


runNum = 0
for perm in permList:
    greened = greenedAcreList[runNum]
    totGreenedAcres = sum(greened)
    runNum += 1
    print "Run Number %s" % runNum  
    print "Total Greened Acres %0.2f" % totGreenedAcres
    print perm
    newRunList=deepcopy(lidList)  # copy the zeroLidList into a new list
    #totalAreaTreated = 0
    for i in range(0,numVariables): 
        #lidNumber = perm[i] # random number of LIDs for this variable
        #subArea = newRunList[i]['SubcatArea'] #area of subcatchment
        newRunList[i]['Number'] = perm[i]  # random number of lids for this variable
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection,perm)
    print "stored this run in mongodb DocId = %s (zero means failed SWMM run)" % newRunDocId
    #cursor = thisRunCollection.find_one({'_id':newRunDocId})

    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"


## RUN ONLY ONCE:  the Zero LID case

In [ ]:
swmmInpFileStr = read_inp_file(swmmInpFileNameStr)
swmmRptDict = swmm_run_return_results(swmmInpFileStr)
print "swmmRptDict type is %s" %  type(swmmRptDict)
zeroLidDocId = thisRunCollection.insert_one(swmmRptDict).inserted_id
print "Stored record %s for zero LID usage" % zeroLidDocId
outflowSeriesCFS = swmmRptDict["outflow_seriesCFS"]
print "Calculating Zero LID Annual CSO Volume"


### Display Zero LID case

In [ ]:
# Show all collections in the database:
# collectionNames = sorted(db.collection_names())
# collectionNames
runs = db[collectionName]
cursor = runs.find()
noLidRun = cursor[0]
print noLidRun['peakCFS']
print noLidRun['precipIN']
print noLidRun['runoffIN']